In [1]:
import time

from bs4 import BeautifulSoup
import pandas as pd
import requests
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager

%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
executable_path = {"executable_path": ChromeDriverManager().install()}
browser = Browser("chrome", **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 93.0.4577
Get LATEST driver version for 93.0.4577
Driver [/Volumes/UsersHome/linglv/.wdm/drivers/chromedriver/mac64/93.0.4577.63/chromedriver] found in cache


<IPython.core.display.Javascript object>

In [135]:
url = "https://www.yelp.com/search?find_desc=Restaurants&find_loc=Georgia%20Tech%2C%20Atlanta%2C%20GA&ns=1&l=g%3A-84.41276550292969%2C33.76816006007425%2C-84.38718795776367%2C33.78956253387451"
browser.visit(url)
html = browser.html
soup = BeautifulSoup(html, "html.parser")

result = []
while True:
    eles = soup.find_all(
        "div", class_=["container__09f24__sxa9-", "hoverable__09f24__3HkY2"]
    )
    for ele in eles:
        name = ele.find_all("div", class_="businessName__09f24__3Ml2X")[0].text
        reviewer = None
        price = None
        restauran_type = {}
        star = None
        print(name)
        try:
            reviewer = ele.find_all(
                "span", class_="reviewCount__09f24__3GsGY css-e81eai"
            )[0].text
        except:
            pass

        try:
            price = ele.find_all("span", class_="priceRange__09f24__2GspP css-e81eai")[
                0
            ].text
        except:
            pass

        try:
            types = ele.find_all("p", class_=["text__09f24__2NHRu", "css-1hx6l2b"])
            l = len(types)
            for i in range(l):
                restauran_type['type_'+types[i].text] = 1
            print(restauran_type)
        except Exception as e:
            print(e)
            pass

        try:
            star = ele.find_all("div", class_="i-stars__09f24___sZu0")[0]["aria-label"]
        except:
            pass
        d = {
            'name': name,
            'reviewer': reviewer,
            'price': price,
            'star': star
        }
        d.update(restauran_type)
        result.append(d)
    try:
        browser.find_by_css(".next-link.navigation-button__09f24__1EzzD.css-166la90").click()
        time.sleep(3)
        print('next page')
        html = browser.html
        soup = BeautifulSoup(html, "html.parser")
    except:
        print('end of the list')
        break


Moe’s Southwest Grill
{'type_Caterers': 1, 'type_Mexican': 1, 'type_Tex-Mex': 1}
Forza Storico
{'type_Beer, Wine & Spirits': 1, 'type_Italian': 1}
1. There on Fifth
{'type_American (New)': 1, 'type_Cocktail Bars': 1, 'type_Chicken Wings': 1}
2. Rock Steady - Atlanta
{'type_Caribbean': 1, 'type_Bars': 1}
3. The Optimist
{'type_Seafood': 1, 'type_Cocktail Bars': 1, 'type_Music Venues': 1}
4. Food Terminal - Atlanta
{'type_Malaysian': 1, 'type_Pan Asian': 1, 'type_Chinese': 1}
5. Forza Storico
{'type_Italian': 1, 'type_Cocktail Bars': 1, 'type_Pasta Shops': 1}
6. Choong Man Chicken - Georgia Tech
{'type_Chicken Wings': 1, 'type_Korean': 1}
7. Pijiu Belly
{'type_Burgers': 1, 'type_Pubs': 1, 'type_Asian Fusion': 1}
8. Velvet Taco
{'type_American (New)': 1, 'type_Tacos': 1, 'type_Vegetarian': 1}
9. JCT Kitchen & Bar
{'type_American (New)': 1, 'type_Bars': 1}
10. Le Fat
{'type_Vietnamese': 1, 'type_Noodles': 1, 'type_Soup': 1}
QDOBA Mexican Eats
{'type_Mexican': 1, 'type_Fast Food': 1}
next p

next page
Moe’s Southwest Grill
{'type_Mexican': 1, 'type_Tex-Mex': 1, 'type_Caterers': 1}
Star of India
{'type_Indian': 1}
101. Goodfellas Pizza and Wings
{'type_Pizza': 1, 'type_Chicken Wings': 1, 'type_Salad': 1}
102. Ray’s Pizza Express
{'type_Pizza': 1}
103. Aerial Kitchen & Bar
{'type_Bars': 1, 'type_American (Traditional)': 1}
104. Papa John’s Pizza
{'type_Pizza': 1}
105. Gyro Bros
{'type_Greek': 1, 'type_American (New)': 1, 'type_Salad': 1}
106. Twisted Taco
{'type_Mexican': 1, 'type_Tex-Mex': 1, 'type_Sports Bars': 1}
107. Genji
{'type_Asian Fusion': 1, 'type_Sushi Bars': 1, 'type_Ramen': 1}
108. Mushi Ni
{'type_Asian Fusion': 1, 'type_American (Traditional)': 1}
109. Auto Spa Bistro
{'type_American (New)': 1, 'type_Car Wash': 1}
110. Firehouse Subs
{'type_Sandwiches': 1, 'type_Fast Food': 1, 'type_Delis': 1}
Forza Storico
{'type_Beer, Wine & Spirits': 1, 'type_Italian': 1}
next page
Moe’s Southwest Grill
{'type_Caterers': 1, 'type_Mexican': 1, 'type_Tex-Mex': 1}
Star of India

<IPython.core.display.Javascript object>

In [136]:
df = pd.DataFrame(result)
df["score"] = df["star"].str.split(" ").str[0]
df["restrant_name"] = df["name"].str.replace("^[0-9]+\.\s", "", regex=True)

<IPython.core.display.Javascript object>

In [137]:
df["advertised_restrant"] = df["name"].str.contains("^[0-9]+\.\s", regex=True)
df.loc[df["advertised_restrant"] == False, "advertised_restrant"] = "yes"
df.loc[df["advertised_restrant"] == True, "advertised_restrant"] = "no"

<IPython.core.display.Javascript object>

In [138]:
new_df = (
    df.sort_values(by=["advertised_restrant"])
    .groupby("restrant_name")
    .last()
    .reset_index()
)

<IPython.core.display.Javascript object>

In [140]:
new_df.to_csv("yelp_clean_data.csv")

<IPython.core.display.Javascript object>